### Step 1 - Get Emoji Dataset

In [1]:
import emoji

In [2]:
# emoji.EMOJI_UNICODE

In [3]:
emoji.emojize(":flexed_biceps:")

'💪'

In [4]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [5]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


### Step 2 - Processing a Custom Dataset

In [6]:
import numpy as np
import pandas as pd

In [7]:
train=pd.read_csv('dataset/train_emoji.csv',header=None)
test=pd.read_csv('dataset/test_emoji.csv',header=None)

In [8]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [8]:
# Let us print the statements with emojis
data=train.values
print(data.shape)

(132, 4)


In [9]:
X_train=train[0]
Y_train=train[1]

In [10]:
X_test=test[0]
Y_test=test[1]

In [11]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴


### Step 3 - Converting Sentences into Embeddings

In [12]:
f=open('glove.6B.50d.txt',encoding='utf-8')

In [13]:
embeddings_index={}

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float')
    embeddings_index[word]=coefs
f.close()

In [14]:
emb_dim=embeddings_index["eat"].shape[0]
print(emb_dim)

50


### Step 4 - Converting Sentences into Vectors(Embedding Layer Output)

In [15]:
def embedding_output(X):
    maxLen=10
    embedding_output=np.zeros((X.shape[0],maxLen,emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        
        for ij in range(len(X[ix])):
            # go to every word in the current sentence
            try:
                embedding_output[ix][ij]=embeddings_index[X[ix][ij].lower()]
            except:
                embedding_output[ix][ij]=np.zeros((50,))
                
    return embedding_output

In [16]:
embeddings_matrix_train=embedding_output(X_train)
embeddings_matrix_test=embedding_output(X_test)

c:\users\k\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
print(X_train[0])
print(len(X_train[0]))

['never', 'talk', 'to', 'me', 'again']
5


In [18]:
print(embeddings_matrix_train.shape)
print(embeddings_matrix_test.shape)

(132, 10, 50)
(56, 10, 50)


In [19]:
from keras.utils import to_categorical

In [20]:
Y_train=to_categorical(Y_train,num_classes=5)
Y_test=to_categorical(Y_test,num_classes=5)
print(Y_train.shape)

(132, 5)


### Step 5 - Define the RNN model

In [21]:
from keras.layers import *
from keras.models import Sequential

In [22]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 64)            29440     
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
____________________________________________________

In [30]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint=ModelCheckpoint("best_model.h5",monitor="val_loss",verbose=True,save_best_only=True)
earlystop=EarlyStopping(monitor='val_acc',patience=10)

hist=model.fit(embeddings_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Epoch 1/100
2/2 [==============================] - 0s 37ms/step - loss: 0.0264 - accuracy: 1.0000 - val_loss: 2.0359 - val_accuracy: 0.6296
Epoch 2/100
2/2 [==============================] - 0s 24ms/step - loss: 0.0153 - accuracy: 1.0000 - val_loss: 2.0355 - val_accuracy: 0.6296
Epoch 3/100
2/2 [==============================] - 0s 24ms/step - loss: 0.0764 - accuracy: 0.9810 - val_loss: 1.8734 - val_accuracy: 0.6667
Epoch 4/100
2/2 [==============================] - 0s 29ms/step - loss: 0.0163 - accuracy: 1.0000 - val_loss: 1.7418 - val_accuracy: 0.6667
Epoch 5/100
2/2 [==============================] - 0s 24ms/step - loss: 0.0218 - accuracy: 1.0000 - val_loss: 1.7164 - val_accuracy: 0.5926
Epoch 6/100
2/2 [==============================] - 0s 28ms/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 1.9435 - val_accuracy: 0.5926
Epoch 7/100
2/2 [==============================] - 0s 24ms/step - loss: 0.0375 - accuracy: 0.9810 - val_loss: 1.8078 - val_accuracy: 0.6296
Epoch 8/100
2/2 [===

2/2 [==============================] - 0s 20ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 1.8273 - val_accuracy: 0.6667
Epoch 60/100
2/2 [==============================] - 0s 20ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 1.8314 - val_accuracy: 0.6667
Epoch 61/100
2/2 [==============================] - 0s 20ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 1.8355 - val_accuracy: 0.6296
Epoch 62/100
2/2 [==============================] - 0s 20ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 1.8409 - val_accuracy: 0.6296
Epoch 63/100
2/2 [==============================] - 0s 20ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 1.8461 - val_accuracy: 0.6296
Epoch 64/100
2/2 [==============================] - 0s 20ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 1.8504 - val_accuracy: 0.6667
Epoch 65/100
2/2 [==============================] - 0s 20ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 1.8628 - val_accuracy: 0.6667
Epoch 66/100
2/2 [========

In [31]:
pred=model.predict_classes(embeddings_matrix_test)

In [32]:
print(pred)

[4 3 3 2 2 2 3 2 4 2 1 2 0 3 1 3 2 2 3 4 3 0 4 2 3 3 2 0 1 2 0 1 3 2 0 1 2
 4 0 2 1 0 0 2 2 2 2 2 0 1 3 0 3 2 3 0]


In [33]:
# model.load_weights("best_model.h5")

In [34]:
model.evaluate(embeddings_matrix_test,Y_test)

2/2 [==============================] - 0s 12ms/step - loss: 2.2099 - accuracy: 0.5714


[2.2098543643951416, 0.5714285969734192]

In [35]:
for i in range(30):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
😁
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
😓
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
😁
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁


# END